# Analysis of different ES implementations

e.g. how to standardize, which sigma to choose

## $\sigma$ in the update step

Is the division by $\sigma$ in Salimans et al correct? **yes**

Is there a difference in std-deviation of the estimates whether we draw from $N(0,1)$ and divide by $\sigma$ or draw from $N(0,\sigma)$ and divide by $\sigma^2$? **--> no**

How to choose $\sigma$ to minimize variance? 
**unclear**, _for each f and x there's some sigma that have lower variance than others. 1/n\_params or a fixed share of mean parameter size seems like a good guess, but this likely depends on normalization of fitness, which seems to be 1 or 2 orders of magnitude more important._

In [7]:
import torch
from torch.distributions.normal import Normal

device = 'cuda'

In [8]:
weights = torch.tensor([2., -2., 5.], device=device)
def f(x):
    return sum([weights[i]*x**i for i in range(len(weights))])

In [9]:
# Analytical Gradient
x = torch.tensor(-1.0, requires_grad=True, device=device)
y = f(x)
y.backward()
x.grad

tensor(-12., device='cuda:0')

In [10]:
sigma = 0.1#float(x)/20+1e-1
npop = int(1e4)

In [11]:
# ES gradient
def es_grad_salimans():
    eps = torch.zeros(npop, device=device).normal_(0, 1)
    fitness = f(x+sigma*eps).detach()
    return (fitness*eps).mean()/sigma

def es_grad_ours():
    eps = torch.zeros(npop, device=device).normal_(0, sigma)
    fitness = f(x+eps).detach()
    return ((fitness)*eps).mean()/sigma**2

In [12]:
theirs = torch.cat([es_grad_salimans().unsqueeze(0) for _ in range(1000)])
ours = torch.cat([es_grad_ours().unsqueeze(0) for _ in range(1000)])

In [13]:
print(sigma)
print(x.grad)
print(theirs.mean(), theirs.std(), theirs.std()/theirs.mean())
print(ours.mean(), ours.std(), ours.std()/ours.mean())

0.1
tensor(-12., device='cuda:0')
tensor(-11.9959, device='cuda:0') tensor(0.9317, device='cuda:0') tensor(-0.0777, device='cuda:0')
tensor(-11.9716, device='cuda:0') tensor(0.9521, device='cuda:0') tensor(-0.0795, device='cuda:0')


## Type or normalization
What's better, reward-scaling or using current utility as baseline?
Does sigma matter?

* **Scaling no longer provides a gradient estimate but just a direction**. For small enough sigma and dividing by sigma **once**, this direction has length ~1.
* Current fitness as baseline vs shifting by the mean has no effect on variance for reasonably small choices of sigma. For very large sigma, the mean provides less variance than the baseline utility. (See caveat below!)
*  However, the mean also seems to give _biased_ estimate for small npop. (Try 10k iterations of x=-0.5, sigma=0.1, npop=100 (or even npop=10). Then gradf=-7, but shifted consistently returns ~-6.9 over 10k iterations of n_pop. It turns out, that's becuase in general $$\mathbb E[\varepsilon F(x+\varepsilon)] \neq 0 $$, so the mean is **not** a valid baseline.

In [15]:
x = torch.tensor(-0., requires_grad=True, device=device)
sigma = 0.05
npop = int(100)
y = f(x)
y.backward()

def es_grad():
    eps = torch.zeros(npop, device=device).normal_(0, sigma)
    fitness = f(x+eps).detach()
    return (fitness*eps).mean()/sigma**2

def es_grad_baseline():
    eps = torch.zeros(npop, device=device).normal_(0, sigma)
    fitness = f(x+eps).detach()
    return ((fitness-y.detach())*eps).mean()/sigma**2

def es_grad_shifted():
    eps = torch.zeros(npop, device=device).normal_(0, sigma)
    fitness = f(x+eps).detach()
    mean = fitness.mean()
    return ((fitness-mean)*eps).mean()/sigma**2

def es_grad_scaled():
    eps = torch.zeros(npop, device=device).normal_(0, sigma)
    fitness = f(x+eps).detach()
    return ((fitness-fitness.mean())/fitness.std()*eps).mean()/sigma

no_base  = torch.cat([es_grad().unsqueeze(0) for _ in range(10000)])
baseline = torch.cat([es_grad_baseline().unsqueeze(0) for _ in range(10000)])
shifted  = torch.cat([es_grad_shifted().unsqueeze(0) for _ in range(10000)])
scaled   = torch.cat([es_grad_scaled().unsqueeze(0) for _ in range(10000)])

print(str('true gradient: \t'), x.grad.cpu())
print(str('no baseline:\t'), no_base.mean(), no_base.std())
print(str('fixed baseline:\t'), baseline.mean(), baseline.std())
print(str('mean baseline:\t'), shifted.mean(), shifted.std(), 'with n-1 in divisor:', shifted.mean() * npop/(npop-1))
print(str('scaled rewards:\t'), scaled.mean(), scaled.std())

true gradient: 	 tensor(-2.)
no baseline:	 tensor(-1.9625, device='cuda:0') tensor(4.0625, device='cuda:0')
fixed baseline:	 tensor(-2.0009, device='cuda:0') tensor(0.2959, device='cuda:0')
mean baseline:	 tensor(-1.9793, device='cuda:0') tensor(0.2896, device='cuda:0') with n-1 in divisor: tensor(-1.9993, device='cuda:0')
scaled rewards:	 tensor(-0.9723, device='cuda:0') tensor(0.0669, device='cuda:0')


In [18]:
def es_wat():
    eps = torch.zeros(npop, device=device).normal_(0, sigma)
    fitness = f(x+eps).detach()
    mean = fitness.mean()
    return mean*eps/sigma**2

In [19]:
npop = int(100)
sigma = 10
wat = torch.cat([es_wat().unsqueeze(0) for _ in range(100_000)])
wat.mean(), wat.std()

(tensor(-0.0232, device='cuda:0'), tensor(51.7255, device='cuda:0'))